In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [14]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()

dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

y = df["Salary"]

X_ = df.drop(["Salary", "League", "Division", "NewLeague"], 
             axis = 1).astype("float64")
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                   random_state=42)

In [22]:
# Verilerin ölçeklendilirmesi
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
# Yapay Sinir Ağı modeli oluşturmaa
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential

ysa_model = Sequential()
ysa_model.add(Dense(units=64, activation="relu", input_dim=X_train.shape[1]))
ysa_model.add(Dense(units=32, activation="relu"))
ysa_model.add(Dense(units=1))

In [26]:
# Modeli derleyin
optimizer = Adam(learning_rate=0.001)
ysa_model.compile(optimizer=optimizer, loss="mean_squared_error")

In [30]:
# Model eğitimi
ysa_model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
7/7 [==============================] - 0s 1ms/step - loss: 104990.3828
Epoch 2/100
7/7 [==============================] - 0s 906us/step - loss: 104583.7500
Epoch 3/100
7/7 [==============================] - 0s 754us/step - loss: 104274.2344
Epoch 4/100
7/7 [==============================] - 0s 662us/step - loss: 103902.9141
Epoch 5/100
7/7 [==============================] - 0s 612us/step - loss: 103511.7188
Epoch 6/100
7/7 [==============================] - 0s 699us/step - loss: 103206.6406
Epoch 7/100
7/7 [==============================] - 0s 602us/step - loss: 102872.4531
Epoch 8/100
7/7 [==============================] - 0s 633us/step - loss: 102542.8359
Epoch 9/100
7/7 [==============================] - 0s 661us/step - loss: 102201.4609
Epoch 10/100
7/7 [==============================] - 0s 606us/step - loss: 101922.5469
Epoch 11/100
7/7 [==============================] - 0s 629us/step - loss: 101592.5469
Epoch 12/100
7/7 [==============================] - 0s 601us/step

7/7 [==============================] - 0s 710us/step - loss: 86976.5078
Epoch 98/100
7/7 [==============================] - 0s 598us/step - loss: 86801.0078
Epoch 99/100
7/7 [==============================] - 0s 608us/step - loss: 86687.9531
Epoch 100/100
7/7 [==============================] - 0s 655us/step - loss: 86636.9844


## Tahmin

In [28]:
# Model tahminleri
y_pred = ysa_model.predict(X_test)

3/3 [==============================] - 0s 762us/step


In [29]:
# Hata ölçümleri
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE: ", np.sqrt(mse))
print("R2: ", r2)

MSE:  355.25331093825605
R2:  0.4170023212144993


## Model Tuning

In [50]:
# Yapay sinir ağı modelini tanımlayın
def create_model(optimizer='adam', units1=64, units2=32, lr=0.001):
    model = Sequential()
    model.add(Dense(units=units1, activation='relu', input_dim=X_train.shape[1])) # Giriş katmanı
    model.add(Dense(units=units2, activation='relu')) # Gizli katman
    model.add(Dense(units=1)) # Çıkış katmanı
    
    if optimizer == 'adam':
        optimizer = Adam(lr=lr) # Adam optimizer ve learning rate'i ayarla
    elif optimizer == 'sgd':
        optimizer = SGD(lr=lr) # SGD optimizer ve learning rate'i ayarla
        
    model.compile(optimizer=optimizer, loss='mse') # Modeli derleme
    return model

# KerasRegressor wrapper kullanarak modeli oluşturun
regressor = KerasRegressor(build_fn=create_model)

# Model tuning için parametre grid'i
param_grid = {
    'optimizer': ['adam', 'sgd'], # optimizer seçenekleri
    'units1': [32, 64, 128], # birinci gizli katmandaki nöron sayısı seçenekleri
    'units2': [32, 64, 128], # ikinci gizli katmandaki nöron sayısı seçenekleri
    'lr': [0.01, 0.001, 0.0001] # learning rate seçenekleri
}

# GridSearchCV kullanarak model tuning'i gerçekleştirin
grid = GridSearchCV(regressor, param_grid, cv=3)
grid.fit(X_train, y_train)

# En iyi parametreleri ve skorları yazdırın
print("En İyi Parametreler: ", grid.best_params_)
print("En İyi R2 Skoru: ", grid.best_score_)

3/3 [==============================] - 0s 740us/step - loss: 543130.8750


/var/folders/t5/k116mkp91b14gpcf_ggn7cn80000gn/T/ipykernel_78609/2088236660.py:17: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = KerasRegressor(build_fn=create_model)


3/3 [==============================] - 0s 737us/step - loss: 355498.4688


7/7 [==============================] - 0s 667us/step - loss: 483067.0312
En İyi Parametreler:  {'lr': 0.01, 'optimizer': 'adam', 'units1': 128, 'units2': 128}
En İyi R2 Skoru:  -472546.25


/Users/batuhanbilgili/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
81 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/batuhanbilgili/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/batuhanbilgili/opt/anaconda3/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 164, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "/var/folders/t5/k116mkp91b14gpcf_ggn7cn80000gn/T/ipykernel_78609/2

In [55]:
optimizer = Adam(learning_rate=0.01)
ysa_model.compile(optimizer=optimizer, loss="mean_squared_error")
ysa_model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

y_pred = ysa_model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE: ", np.sqrt(mse))
print("R2: ", r2)

Epoch 1/100
7/7 [==============================] - 0s 616us/step - loss: 490515.9688
Epoch 2/100
7/7 [==============================] - 0s 717us/step - loss: 472831.7500
Epoch 3/100
7/7 [==============================] - 0s 706us/step - loss: 429514.5625
Epoch 4/100
7/7 [==============================] - 0s 680us/step - loss: 342566.2812
Epoch 5/100
7/7 [==============================] - 0s 670us/step - loss: 220665.4375
Epoch 6/100
7/7 [==============================] - 0s 796us/step - loss: 155780.3438
Epoch 7/100
7/7 [==============================] - 0s 780us/step - loss: 147925.3438
Epoch 8/100
7/7 [==============================] - 0s 730us/step - loss: 139567.3125
Epoch 9/100
7/7 [==============================] - 0s 620us/step - loss: 118520.0781
Epoch 10/100
7/7 [==============================] - 0s 750us/step - loss: 112516.6562
Epoch 11/100
7/7 [==============================] - 0s 728us/step - loss: 108368.9609
Epoch 12/100
7/7 [==============================] - 0s 636us/st

7/7 [==============================] - 0s 597us/step - loss: 66671.9844
Epoch 98/100
7/7 [==============================] - 0s 605us/step - loss: 65339.3750
Epoch 99/100
7/7 [==============================] - 0s 576us/step - loss: 65812.2344
Epoch 100/100
3/3 [==============================] - 0s 644us/step
MSE:  355.3377564947648
R2:  0.4167251251225196
